In [1]:
import sys
import os


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px


package_path = os.path.abspath(os.path.join(os.getcwd(), "../../"))

if package_path not in sys.path:
    sys.path.append(package_path)

from flair.prophet_evaluator import ProphetEvaluator

/Users/franperic/projects/learnings/flair/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prophet = ProphetEvaluator(
    df=pd.DataFrame(
        {
            "date": pd.date_range("2005-01-01", periods=1000, freq="M"),
            "sales": np.random.normal(size=1000),
        }
    ),
    target="sales",
    date="date",
    testtype="structural",
)
prophet.evaluate()

 simulation runs: 100%|██████████| 3/3 [00:02<00:00,  1.08it/s]


In [3]:
prophet.plot_interventions()

/Users/franperic/projects/learnings/flair/.venv/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [4]:
df = pd.read_parquet("../../data/Monthly-train.parquet")
info = pd.read_csv("../../data/info.csv")
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V2786,V2787,V2788,V2789,V2790,V2791,V2792,V2793,V2794,V2795
0,M1,8000.0,8350.0,8570.0,7700.0,7080.0,6520.0,6070.0,6650.0,6830.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M2,2440.0,2490.0,2710.0,2290.0,1960.0,1990.0,1960.0,2530.0,2640.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M3,2670.0,2590.0,2784.0,2816.0,2888.0,2906.0,2839.0,2860.0,2798.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M4,7264.0,7215.0,7022.0,7040.0,6966.0,6984.0,7025.0,7082.0,6956.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M5,4210.0,4290.0,4250.0,4320.0,4320.0,4330.0,4320.0,4260.0,4220.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
startdate_ts = pd.to_datetime("1963-01-01")

In [6]:
ts_id = "M54"
meta_ts = info.loc[info.M4id == ts_id]
# startdate_ts = pd.to_datetime(meta_ts.StartingDate.values[0])
ts = df.loc[df.V1 == ts_id].T.dropna().values[1:]
ts = ts.flatten()
ts = pd.DataFrame({"ds": pd.date_range(startdate_ts.date(), periods=len(ts), freq="M"), "y": ts})
ts

,ds,y
0,1963-01-31,5910.0
1,1963-02-28,4640.0
2,1963-03-31,4610.0
3,1963-04-30,6050.0
4,1963-05-31,5860.0
...,...,...
624,2015-01-31,5240.0
625,2015-02-28,5490.0
626,2015-03-31,4900.0
627,2015-04-30,5000.0


In [21]:
fig = px.line(x=ts.ds, y=ts.y, color_discrete_sequence=["#4561a8"])
fig.update_layout(title="Example Time Series M54", xaxis_title="Date", yaxis_title="Value", width=700, height=400)
fig.write_html("ts.html")
fig

/Users/franperic/projects/learnings/flair/.venv/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Interventions Overview

In [7]:
# %%
evaluator = ProphetEvaluator(ts, "y", "ds", "structural", sim_runs=10)
evaluator.evaluate()

 simulation runs: 100%|██████████| 10/10 [00:14<00:00,  1.40s/it]


In [16]:
fig = evaluator.plot_interventions()
fig.update_layout(width=700, height=2900)
fig.write_html("interventions.html")
fig

/Users/franperic/projects/learnings/flair/.venv/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Global Results

In [ ]:
evaluator.plot_summary()

## Detailed View

In [ ]:
evaluator.plot_detail(run = 1)

## MAPES

In [ ]:
evaluator.plot_mapes(run=0)